In [1]:
%run ./__init__edwh__notebooks.ipynb

Python 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:20:46) 
[GCC 9.4.0]
 * postgresql://postgres@pgpool:5432/backend
Done.


[source](https://pypi.org/project/ipython-sql/)    
The << operator captures query results in a local variable, and can be used in multi-line `%%sql`:

```
In [19]: %%sql works << SELECT title, year
    ...: FROM work
    ...:
43 rows affected.
```


In [42]:
%%sql teams_per_item << with items_with_short_sd as (
    select item.gid,
       substring(short_description from '\[.*?\]\(.*?\)') as first_link,
       short_description  as body from item
  inner join "user" u on u.gid = item.author and u.email not like '%@roc.nl' and u.email = 'eddie@educationwarehouse.nl'
where item.platform = 'SvS')
select gid
       ,first_link
       ,substring(first_link from '\[(.*?)\]') as org
       ,substring(first_link from '\((.*?)\)') as web
       --,body
from items_with_short_sd
where first_link is not null
  and position(first_link in body) < position('.' in body)
order by 3

 * postgresql://postgres@pgpool:5432/backend
636 rows affected.
Returning data to local variable teams_per_item


In [43]:
unique_team_names = {team[2].strip():team[3] for team in teams_per_item}

In [44]:
unique_team_names

{'2Samen': 'https://www.2samen.nl',
 'Aares VMBO': 'https://www.aeresvmbo.nl/ede',
 'Academie Mens en Maatschappij (AMM) van Saxion': 'AMM',
 'ACBS Wereldwijs': 'https://www.wereldwijs.nu/',
 'Agnieten College': 'https://www.agnietennieuwleusen.nl/',
 'Alan Turingschool': 'https://www2.askoscholen.nl/scholen/alanturingschool/Paginas/default.aspx',
 'Alexander Roozendaalschool': 'https://www.roozendaalschool.nl/',
 'Alfa-college in Hardenberg': 'https://www.alfa-college.nl/opleidingen-en-cursussen/studiekeuze-events?gclid=EAIaIQobChMI4uH-0-m38AIVBfhRCh1dcgZGEAAYASAAEgJ3M_D_BwE',
 'Alkwin Kollege': 'https://alkwin.nl/',
 'Almere College': 'https://www.almerecollege.nl',
 'Amadeus Lyceum': 'https://amadeuslyceum.nl/',
 'Amalia Astroschool voor Christelijk Basisonderwijs': 'https://www.amalia-astro.nl/',
 'Anna Lyceum': 'https://annavanrijn.nl/scholen/anna-lyceum/',
 'Anna van Rijn College': 'https://annavanrijn.nl/',
 'Antoniuscollege': 'https://www.antoniuscollegegouda.nl/home',
 'Antoni

In [50]:
import uuid
team_name_to_user_id = {}
for team,web in unique_team_names.items(): 
    print(team,web)
    team_name_to_user_id[team] = db.user.insert(platform='SvS', email='contactknop@educationwarehouse.nl', name='Team '+team, user_provided_organisation=team, has_validated_email=True, gid=uuid.uuid4())

2Samen https://www.2samen.nl
Aares VMBO https://www.aeresvmbo.nl/ede
Academie Mens en Maatschappij (AMM) van Saxion AMM
ACBS Wereldwijs https://www.wereldwijs.nu/
Agnieten College https://www.agnietennieuwleusen.nl/
Alan Turingschool https://www2.askoscholen.nl/scholen/alanturingschool/Paginas/default.aspx
Alexander Roozendaalschool https://www.roozendaalschool.nl/
Alfa-college in Hardenberg https://www.alfa-college.nl/opleidingen-en-cursussen/studiekeuze-events?gclid=EAIaIQobChMI4uH-0-m38AIVBfhRCh1dcgZGEAAYASAAEgJ3M_D_BwE
Alkwin Kollege https://alkwin.nl/
Almere College https://www.almerecollege.nl
Amadeus Lyceum https://amadeuslyceum.nl/
Amalia Astroschool voor Christelijk Basisonderwijs https://www.amalia-astro.nl/
Anna Lyceum https://annavanrijn.nl/scholen/anna-lyceum/
Anna van Rijn College https://annavanrijn.nl/
Antoniuscollege https://www.antoniuscollegegouda.nl/home
Antoniuscollege Gouda https://www.antoniuscollegegouda.nl/home
Antoon Schellenscollege https://www.antoonschellen

In [52]:
team_name_to_user_id

{'2Samen': 4530,
 'Aares VMBO': 4531,
 'Academie Mens en Maatschappij (AMM) van Saxion': 4532,
 'ACBS Wereldwijs': 4533,
 'Agnieten College': 4534,
 'Alan Turingschool': 4535,
 'Alexander Roozendaalschool': 4536,
 'Alfa-college in Hardenberg': 4537,
 'Alkwin Kollege': 4538,
 'Almere College': 4539,
 'Amadeus Lyceum': 4540,
 'Amalia Astroschool voor Christelijk Basisonderwijs': 4541,
 'Anna Lyceum': 4542,
 'Anna van Rijn College': 4543,
 'Antoniuscollege': 4544,
 'Antoniuscollege Gouda': 4545,
 'Antoon Schellenscollege': 4546,
 'Arentheem College': 4547,
 'Arentheem College vestiging Thomas a Kempis': 4548,
 'Ariensschool': 4549,
 'Arte College': 4550,
 'Ashram College': 4551,
 'Augustinianum': 4552,
 'basisschool De Bellefleur': 4553,
 'basisschool De Bijenkorf': 4554,
 'basisschool De Branding in Swifterbant': 4555,
 'basisschool de Driesprong': 4556,
 'basisschool de Griftschool': 4557,
 'basisschool De Hoeksteen': 4558,
 'basisschool De Hofvijver': 4559,
 'basisschool De Koperwiek':

In [51]:
db.commit()

In [34]:
# for record in team_name_to_user_id.values():
#     db(db.user.id == record).delete()

In [60]:
for item_gid, _, team_name, *_ in teams_per_item:
    team_for_item = db(db.user.name == 'Team '+team_name.strip()).select().first()
    item = db.item(gid=item_gid)
    print(item.author, team_for_item.gid)

c8831058-34a3-42f1-ad83-eaf84aef2a30 95f74046-ea82-4fe7-ba16-beefcd28ba0d
c8831058-34a3-42f1-ad83-eaf84aef2a30 46637b19-6149-494d-af61-5922773df0c6
c8831058-34a3-42f1-ad83-eaf84aef2a30 49a097da-acae-4aed-9013-cbb479d9c8e4
c8831058-34a3-42f1-ad83-eaf84aef2a30 ad2f5a6d-ebef-4267-a2ad-a583877aef75
c8831058-34a3-42f1-ad83-eaf84aef2a30 6652bfa8-cd4f-4866-8a03-23ab97e0a4a7
c8831058-34a3-42f1-ad83-eaf84aef2a30 6652bfa8-cd4f-4866-8a03-23ab97e0a4a7
c8831058-34a3-42f1-ad83-eaf84aef2a30 6652bfa8-cd4f-4866-8a03-23ab97e0a4a7
c8831058-34a3-42f1-ad83-eaf84aef2a30 8f5dfd15-da4a-4443-8115-d0d8b5f40d4f
c8831058-34a3-42f1-ad83-eaf84aef2a30 3ae9a2a6-9bec-4c9d-a7e7-0ad98538b055
c8831058-34a3-42f1-ad83-eaf84aef2a30 3ae9a2a6-9bec-4c9d-a7e7-0ad98538b055
c8831058-34a3-42f1-ad83-eaf84aef2a30 3ae9a2a6-9bec-4c9d-a7e7-0ad98538b055
c8831058-34a3-42f1-ad83-eaf84aef2a30 3941d125-049d-4f0c-b36f-23281280e0f6
c8831058-34a3-42f1-ad83-eaf84aef2a30 889ed0c8-0a2e-47a0-a738-f4cdb8fb5d6f
c8831058-34a3-42f1-ad83-eaf84aef2a30 4